In [8]:
import copy
import logging

import numpy as np
import pandas as pd

In [9]:
def compute_entropy(y_class):
    """计算信息熵"""
    y_unique = np.unique(y_class)
    if y_unique.shape[0] == 1:  # 只有一个类别, 熵为0
        return 0.
    ety = 0.
    for i in range(len(y_unique)):  # 取每个类别
        p = np.sum(y_class == y_unique[i]) / len(y_class)
        ety += p * np.log2(p)
    return -ety


def compute_condition_entropy(X_feature, y_class):
    """计算条件熵"""
    f_unique = np.unique(X_feature)
    result = 0.
    for i in range(len(f_unique)):
        index_x = (X_feature == f_unique[i])
        p = np.sum(index_x) / len(y_class)
        ety = compute_entropy(y_class[index_x])
        result += p * ety
    return result


def compute_cost_in_leaf(labels):
    """
    计算节点的损失
    """
    y_count = np.bincount(labels)
    n_samples = len(labels)
    cost = 0
    # 计算整个节点内部的损失值
    for i in range(len(y_count)):
        if y_count[i] == 0:
            continue
        cost += y_count[i] * np.log2(y_count[i] / n_samples)
    return -cost


def get_label(labels):
    """
    根据给定标签，返回出现次数最多的类别
    """
    r = {}
    for i in range(len(labels)):
        r[labels[i]] = r.setdefault(labels[i], 0) + 1
    return sorted(r.items(), key=lambda x: x[1])[-1][0]

In [10]:
class Node(object):
    def __init__(self, ):
        self.sample_index = None  # 保存当前节点中对应样本在数据集中的索引
        self.values = None  # 保存每个类别的数量
        self.features = None  # 保存当前节点状态时特征集中剩余特征
        self.feature_id = -1  # 保存当前节点对应划分特征的id
        self.label = None  # 保存当前节点对应的类别标签（叶子节点才有）
        self.n_samples = 0  # 保存当前节点对应的样本数量
        self.children = {}  # 保存当前节点对应的孩子节点
        self.criterion_value = 0.
        self.n_leaf = 0  # 以当前节点为根节点时其叶子节点的个数
        self.leaf_costs = 0.

    def __str__(self):
        return f"<======================>\n" \
               f"当前节点所有样本的索引({self.sample_index})\n" \
               f"当前节点的样本数量({self.n_samples})\n" \
               f"当前节点每个类别的样本数({self.values})\n" \
               f"当前节点对应的信息增益（比）({round(self.criterion_value, 3)})\n" \
               f"当前节点状态时特征集中剩余特征({self.features})\n" \
               f"当前节点状态时划分特征ID({self.feature_id})\n" \
               f"当前节点对应的类别标签为({self.label})\n" \
               f"当前节点为根节点对应孩子节点数为({self.n_leaf})\n" \
               f"当前节点为根节点对应孩子节点损失为({self.leaf_costs})\n" \
               f"当前节点对应的孩子为({self.children.keys()})"

In [11]:
class DecisionTree(object):
    def __init__(self, min_samples_split=2, criterion='id3', epsilon=1e-5, alpha=0.):
        self._X = None
        self.n_classes = None
        self._y = None
        self.root = None
        self.min_samples_split = min_samples_split  # 用来控制是否停止分裂
        self.epsilon = epsilon  # 用来控制是否停止分裂
        self.criterion = criterion  # 划分标注，ID3还是C4.5
        self.alpha = alpha

    def fit(self, X, y):
        """训练构建决策树"""
        self._y = np.array(y).reshape(-1)
        self.n_classes = len(np.bincount(y))  # 得到当前数据集的类别数量
        feature_ids = [i for i in range(X.shape[1])]  # 得到特征的序号
        self._X = np.hstack(([X, np.arange(len(X)).reshape(-1, 1)]))
        self._build_tree(self._X, feature_ids)  # 递归构建决策树
        self._pruning_leaf()
        return self

    def _predict_one_sample(self, x):
        """
        预测单一样本
        """
        current_node = self.root
        while True:
            current_feature_id = current_node.feature_id
            current_feature = x[current_feature_id]

            # 当前节点为叶子节点，或者由于数据集不充分当前节点的孩子节点不存在下一个划分节点的某一个取值
            if len(current_node.children) < 1 or current_feature not in current_node.children:
                return current_node.values
            current_node = current_node.children[current_feature]

    def predict(self, X):
        """
        预测样本
        """
        results = []
        for x in X:
            results.append(self._predict_one_sample(x))
        results = np.array(results)
        logging.debug(f"原始预测结果为:\n{results}")
        y_pred = np.argmax(results, axis=1)
        return y_pred

    def score(self, X, y):
        """计算样本在测试集上准确率"""
        pred = dt.predict(X)
        print(pred)
        score = 0
        for i in range(len(pred)):
            if pred[i] == y[i]:
                score += 1

        return score / len(pred)

    def _split_criterion(self, ety, X_feature, y_class):
        """返回不同分类决策下的选择"""
        c_ety = compute_condition_entropy(X_feature, y_class)
        info_gains = ety - c_ety  # 计算信息增益
        if self.criterion == "id3":
            return info_gains
        elif self.criterion == "c45":
            f_ety = compute_entropy(X_feature)
            if f_ety == 0:
                return 0
            else:
                return info_gains / f_ety  # 计算信息增益率
        else:
            raise ValueError(f"划分标准 self.criterion = {self.criterion}只能是 id3 和 c45 其中之一！")

    def _build_tree(self, data, f_ids):
        """构建决策树"""
        x_ids = np.array(data[:, -1], dtype=int).reshape(-1)  # 得到每个样本在原始数据集中的索引
        labels = self._y[x_ids]

        # 定义当前节点并保存对应的节点信息
        node = Node()
        node.sample_index = x_ids
        node.n_samples = len(labels)
        node.values = np.bincount(labels, minlength=self.n_classes)
        node.features = f_ids
        # 为空则将当前节点作为根节点
        if self.root is None:
            self.root = node

        y_unique = np.unique(labels)
        # 最小样本数的判断条件
        if y_unique.shape[0] == 1 or len(f_ids) < 1 or node.n_samples <= self.min_samples_split:
            node.label = get_label(labels)  # 确定当前节点对应的类别
            return node
        ety = compute_entropy(labels)  # 计算当前节点所有样本对应的信息熵
        node.criterion_value = ety

        max_criterion = 0
        best_feature_id = -1
        # 遍历当前节点可进行选择的划分特征，然后计算各个特征划分下的信息增益（比）来选择最佳特征
        for id in f_ids:
            criterion = self._split_criterion(ety, data[:, id], labels)
            if criterion > max_criterion:
                max_criterion = criterion
                best_feature_id = id
        # 当前计算得到的最大划分指标是否小于阈值，如果是则直接返回当前节点
        if max_criterion < self.epsilon:
            node.label = get_label(labels)
            return node
        node.feature_id = best_feature_id
        feature_values = np.unique(data[:, best_feature_id])  # 取最佳划分特征中特征的取值情况
        candidate_ids = copy.copy(f_ids)  # f_ids是一个列表传入函数_build_tree时本质上是传入的地址，因此需要先进行复制
        candidate_ids.remove(best_feature_id)
        # 根据当前特征维度的取值，来取对应的样本索引，并递归的进行节点划分
        for f in feature_values:
            c = data[:, best_feature_id] == f
            index = np.array([i for i in range(len(c)) if c[i] == True])
            node.children[f] = self._build_tree(data[index], candidate_ids)
        return node

    def level_order(self, return_node=False):
        """进行决策树的遍历"""
        root = self.root
        if not root:
            return []
        queue = [root]
        res = []
        # 利用队列遍历所有节点
        while queue:
            tmp = []
            for i in range(len(queue)):
                node = queue.pop(0)
                tmp.append(node)
                for k, v in node.children.items():
                    queue.append(v)
            res.append(tmp)
        if return_node:
            return res  # 返回所有层次遍历的结果(各层节点的地址)

        # 直接输出整个层次遍历的结果
        for i, r in enumerate(res):
            logging.debug(f"第{i + 1}层的节点为：")
            for node in r:
                logging.debug(node)

    def _pruning_leaf(self):
        """进行剪枝"""
        level_order_nodes = self.level_order(return_node=True)
        # 获取得到层次遍历的所有结果
        logging.debug(f"正在进行剪枝操作……")
        for i in range(len(level_order_nodes) - 1, -1, -1):
            # 从下往上依次遍历每一层节点
            current_level_nodes = level_order_nodes[i]  # 取第i层的所有节点
            for j in range(len(current_level_nodes)):
                current_node = current_level_nodes[j]  # 取第i层的第j个节点
                if len(current_node.children) == 0:  # 当前节点为叶子节点时
                    current_node.n_leaf = 1  # 令其叶节点个数为1
                else:
                    for _, leaf_node in current_node.children.items():
                        current_node.n_leaf += leaf_node.n_leaf  # 统计以当前节点为根节点时的叶子节点数量
                if self._is_pruning_leaf(current_node):
                    current_node.children = {}
                    current_node.n_leaf = 1

    def _is_pruning_leaf(self, node):
        """
        判断是否对当前节点进行剪枝
        """
        if len(node.children) < 1:  # 当前节点为叶子节点时，计算叶子节点对应的损失
            node.leaf_costs = compute_cost_in_leaf(self._y[node.sample_index])
            return False
        parent_cost = compute_cost_in_leaf(self._y[node.sample_index])  # 剪枝后的损失
        for (_, leaf_node) in node.children.items():  # 剪枝前累加所有叶子节点的损失
            node.leaf_costs += leaf_node.leaf_costs
        # logging.debug(f"当前节点的损失为：{parent_cost} + {self.alpha}")
        # logging.debug(f"当前节点的孩子节点损失和为：{node.leaf_costs} + {self.alpha} * {node.n_leaf}")

        #  当剪枝前的损失  >  剪枝后的损失， 则表示当前节点可以进行剪枝（减掉其所有孩子）
        if node.leaf_costs + self.alpha * node.n_leaf > parent_cost + self.alpha:
            return True
        return False

In [12]:
class FeatureDiscretization(object):
    def __init__(self, continuous_columns):
        self.continuous_columns = continuous_columns

    def fit(self, x, y):
        for col_name in self.continuous_columns:
            col = x[col_name].to_numpy()
            col_sorted = np.sort(col)
            dividing_point_list = []
            for i in np.arange(0, len(col_sorted) - 1):
                avg = np.average([col_sorted[i], col_sorted[i + 1]])
                if avg not in dividing_point_list:
                    dividing_point_list.append(avg)
            dividing_value = self.get_dividing_value(col, dividing_point_list, y)
            if dividing_value is not None:
                col_new = col.copy()
                col_new[col >= dividing_value] = 1
                col_new[col < dividing_value] = 0
                x[col_name] = col_new
        return x

    def get_dividing_value(self, col, dividing_point_list, y):
        gain_list = []
        for value in dividing_point_list:
            col_new = col.copy()
            col_new[col >= value] = 1
            col_new[col < value] = 0

            # 计算信息增益率
            ety = compute_entropy(col_new)
            if ety == 0:
                gain_list.append(0)
            else:
                gain_list.append((compute_entropy(y) - compute_condition_entropy(col_new, y)) / ety)
        return dividing_point_list[np.argmax(gain_list)] if len(gain_list) != 0 else None

In [13]:
data = pd.read_csv('data/german_clean.csv')
x = data.iloc[:, :20]
y = data.iloc[:, 20:].to_numpy().flatten()

# 连续数据离散化
continuous_columns = ['duration_new', 'credit_amount', 'age']
transformer = FeatureDiscretization(continuous_columns=continuous_columns)
x = transformer.fit(x, y)

# 训练模型
dt = DecisionTree(criterion='c45')
dt.fit(x.to_numpy(), y)
nodes = dt.level_order(return_node=True)  # 剪枝

dt.score(x.to_numpy(), y)

[1 2 1 1 2 1 1 1 1 2 2 2 1 2 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1
 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 1 1 1
 2 1 2 1 1 1 2 1 1 1 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1
 1 1 2 1 1 2 1 1 1 2 1 1 1 2 1 1 1 1 2 1 2 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 1 1 1 1 1 2 2 2 1 2
 1 2 1 2 1 2 2 2 1 1 2 1 2 1 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 1 1 1 2 1 2 1 1 1 1 1 1 2 1 1 2 1 1 1 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 2
 1 1 1 1 1 2 2 1 2 1 1 2 2 1 1 1 1 2 1 2 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 1 2 1 2 1 2 1 1 1 1 2 1 1 1 2 1
 1 1 1 1 1 2 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 2 1
 1 1 2 1 1 2 1 2 1 2 1 1 2 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 2 1 1 1 2
 2 1 2 1 1 2 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 2 2 2 1 2 1 1 1 1 1 1
 1 1 1 1 2 1 1 1 1 1 2 1 

0.973